### Import Labraries

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

### Load Features

In [9]:
features_df = pd.read_csv("../Feature_WM_Level/Feature_S2_Cheese_C1_GTEA.csv")
action_classes = sorted(features_df["Action_class"].unique())
action_mapping = {action_classes[i]: i for i in range(len(action_classes))}
features_df["action_class_mapped"] = features_df["Action_class"].map(action_mapping)
train_df, test_df = train_test_split(features_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
def process_data(df):
    X = torch.tensor(df.iloc[:, 3:].values, dtype=torch.float32)
    y = torch.tensor(df["action_class_mapped"].values, dtype=torch.long)
    return X, y

X_train, y_train = process_data(train_df)
X_val, y_val = process_data(val_df)
X_test, y_test = process_data(test_df)

### SVSG Graph using Dynamic Percentile Threhold +

In [10]:
def dynamic_percentile_graph(X, percentile=98, device='cuda'):
    X = X.to(device)
    N = X.shape[0]
    sim_matrix = F.cosine_similarity(X.unsqueeze(1), X.unsqueeze(0), dim=2)
    threshold = torch.quantile(sim_matrix, percentile / 100)
    adj_matrix = (sim_matrix >= threshold).float()
    adj_matrix.fill_diagonal_(0)
    row_indices, col_indices = torch.nonzero(adj_matrix, as_tuple=True)
    edge_index = torch.stack([row_indices, col_indices], dim=0)
    return edge_index

# GCN Model
class GCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Multi-Task Loss
class MultiTaskLoss(nn.Module):
    def __init__(self, alpha=0.8):
        super(MultiTaskLoss, self).__init__()
        self.ce_loss = nn.CrossEntropyLoss()
        self.alpha = alpha

    def forward(self, outputs, targets, features):
        ce = self.ce_loss(outputs, targets)
        temporal_loss = torch.mean(torch.abs(features[1:] - features[:-1]))
        return self.alpha * ce + (1 - self.alpha) * temporal_loss



In [11]:
# Model Initialization
input_dim = X_train.shape[1]
hidden_dim = 128
output_dim = features_df['action_class_mapped'].nunique()
model = GCNModel(input_dim, hidden_dim, output_dim)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and Loss
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
criterion = MultiTaskLoss()



In [12]:
# Training Loop
from sklearn.metrics import precision_score, recall_score, f1_score
num_epochs = 100
percentile_threshold = 98

for epoch in range(num_epochs):
    model.train()
    edge_index = dynamic_percentile_graph(X_train, percentile=percentile_threshold).to(device)
    optimizer.zero_grad()
    output = model(X_train.to(device), edge_index)
    loss = criterion(output, y_train.to(device), X_train.to(device))
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        edge_index_val = dynamic_percentile_graph(X_val, percentile=percentile_threshold).to(device)
        val_output = model(X_val.to(device), edge_index_val)
        val_loss = criterion(val_output, y_val.to(device), X_val.to(device))
        val_probs = torch.softmax(val_output, dim=1)
        val_preds = val_probs.argmax(dim=1).cpu().numpy()

        top1_correct = (val_preds == y_val.cpu().numpy()).sum()
        top1_acc = top1_correct / len(y_val)

        precision = precision_score(y_val.cpu().numpy(), val_preds, average='weighted', zero_division=1)
        recall = recall_score(y_val.cpu().numpy(), val_preds, average='weighted', zero_division=1)
        f1 = f1_score(y_val.cpu().numpy(), val_preds, average='weighted', zero_division=1)

        top5_preds = torch.topk(val_probs, 5, dim=1).indices.cpu().numpy()
        top5_correct = sum([y in top5 for y, top5 in zip(y_val.cpu().numpy(), top5_preds)])
        top5_acc = top5_correct / len(y_val)

        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Loss: {loss.item():.4f} - "
              f"Val Loss: {val_loss.item():.4f} - "
              f"Top-1 Acc: {top1_acc:.4f} - "
              f"Top-5 Acc: {top5_acc:.4f} - "
              f"Precision: {precision:.4f} - "
              f"Recall: {recall:.4f} - "
              f"F1-Score: {f1:.4f}")


Epoch 1/100 - Loss: 2.7824 - Val Loss: 2.4227 - Top-1 Acc: 0.0196 - Top-5 Acc: 0.2745 - Precision: 0.9711 - Recall: 0.0196 - F1-Score: 0.0009
Epoch 2/100 - Loss: 2.4096 - Val Loss: 2.0888 - Top-1 Acc: 0.0294 - Top-5 Acc: 0.4608 - Precision: 0.8975 - Recall: 0.0294 - F1-Score: 0.0182
Epoch 3/100 - Loss: 2.0647 - Val Loss: 1.7941 - Top-1 Acc: 0.0686 - Top-5 Acc: 0.7549 - Precision: 0.4774 - Recall: 0.0686 - F1-Score: 0.0835
Epoch 4/100 - Loss: 1.7606 - Val Loss: 1.5511 - Top-1 Acc: 0.3824 - Top-5 Acc: 0.8824 - Precision: 0.5747 - Recall: 0.3824 - F1-Score: 0.3561
Epoch 5/100 - Loss: 1.5092 - Val Loss: 1.3735 - Top-1 Acc: 0.5196 - Top-5 Acc: 0.8431 - Precision: 0.7232 - Recall: 0.5196 - F1-Score: 0.4034
Epoch 6/100 - Loss: 1.3235 - Val Loss: 1.2675 - Top-1 Acc: 0.5882 - Top-5 Acc: 0.8431 - Precision: 0.7578 - Recall: 0.5882 - F1-Score: 0.4357
Epoch 7/100 - Loss: 1.2082 - Val Loss: 1.2205 - Top-1 Acc: 0.5882 - Top-5 Acc: 0.8529 - Precision: 0.7578 - Recall: 0.5882 - F1-Score: 0.4357
Epoch 

Epoch 61/100 - Loss: 0.5883 - Val Loss: 0.6705 - Top-1 Acc: 0.7255 - Top-5 Acc: 0.9706 - Precision: 0.7938 - Recall: 0.7255 - F1-Score: 0.6309
Epoch 62/100 - Loss: 0.5819 - Val Loss: 0.6648 - Top-1 Acc: 0.7255 - Top-5 Acc: 0.9706 - Precision: 0.7938 - Recall: 0.7255 - F1-Score: 0.6309
Epoch 63/100 - Loss: 0.5756 - Val Loss: 0.6592 - Top-1 Acc: 0.7255 - Top-5 Acc: 0.9608 - Precision: 0.7938 - Recall: 0.7255 - F1-Score: 0.6309
Epoch 64/100 - Loss: 0.5695 - Val Loss: 0.6535 - Top-1 Acc: 0.7255 - Top-5 Acc: 0.9608 - Precision: 0.7938 - Recall: 0.7255 - F1-Score: 0.6309
Epoch 65/100 - Loss: 0.5633 - Val Loss: 0.6479 - Top-1 Acc: 0.7255 - Top-5 Acc: 0.9608 - Precision: 0.7938 - Recall: 0.7255 - F1-Score: 0.6309
Epoch 66/100 - Loss: 0.5573 - Val Loss: 0.6423 - Top-1 Acc: 0.7255 - Top-5 Acc: 0.9608 - Precision: 0.7938 - Recall: 0.7255 - F1-Score: 0.6309
Epoch 67/100 - Loss: 0.5513 - Val Loss: 0.6368 - Top-1 Acc: 0.7255 - Top-5 Acc: 0.9608 - Precision: 0.7938 - Recall: 0.7255 - F1-Score: 0.6309

In [ ]:

# Final Confusion Matrix
if epoch == num_epochs - 1:
    conf_matrix = confusion_matrix(y_val.cpu().numpy(), val_preds)
    plt.figure(figsize=(10, 5))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix - Validation Set')
    plt.show()


In [ ]:

# Save the model
saved_model_path = "../SavedModels/best_model_gtea.pth"
torch.save(model.state_dict(), saved_model_path)
print(f"Model saved at: {saved_model_path}")

action_label = ['take plate', 'open bin', 'throw leftover', 'close bin', 'put-down plate', 'take sponge', 'squeeze sponge', 'open tap', 
                'rinse sponge', 'take-up liquid', 'pour-up liquid', 'take fork', 'wash fork', 'take knife', 'wash knife', 'put-down knife', 'wash plate','take spatula','rinse spatula','put-down spatula','take colander']
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Reds',
            xticklabels=action_label, yticklabels=action_label)